# Julia Set

### Copyright Information

In [ ]:
# Copyright (c) 2024-2025 Ben Ashbaugh
#
# SPDX-License-Identifier: MIT

## Sample Purpose

This is another sample that generates a fractal image.
It uses an OpenCL kernel to compute a [Julia set](https://en.wikipedia.org/wiki/Julia_set) image, which is displayed in this notebook and then written to a BMP file.

This sample also shows how to time the execution of an OpenCL kernel, and how to pass a different local work group size to change the way the kernel is executed.

## Sample

To start the sample, we will import pyopencl and a few other packages that this sample uses.

In [ ]:
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import pyopencl as cl
import PIL
import time

We will then define the size of the image we want to generate and we will write our Julia kernel.
Each OpenCL work-item computes one element of the set, or equivalently, one pixel in the output image.

In [ ]:
width = 512
height = 512

kernelString = """
kernel void Julia( global uchar4* dst, float cr, float ci )
{
    const float cMinX = -1.5f;
    const float cMaxX =  1.5f;
    const float cMinY = -1.5f;
    const float cMaxY =  1.5f;

    const int cWidth = get_global_size(0);
    const int cIterations = 16;

    int x = (int)get_global_id(0);
    int y = (int)get_global_id(1);

    float a = x * ( cMaxX - cMinX ) / cWidth + cMinX;
    float b = y * ( cMaxY - cMinY ) / cWidth + cMinY;

    float result = 0.0f;
    const float thresholdSquared = cIterations * cIterations / 64.0f;

    for( int i = 0; i < cIterations; i++ ) {
        float aa = a * a;
        float bb = b * b;

        float magnitudeSquared = aa + bb;
        if( magnitudeSquared >= thresholdSquared ) {
            break;
        }

        result += 1.0f / cIterations;
        b = 2 * a * b + ci;
        a = aa - bb + cr;
    }

    result = max( result, 0.0f );
    result = min( result, 1.0f );

    // RGBA
    float4 color = (float4)( result, sqrt(result), 1.0f, 1.0f );

    dst[ y * cWidth + x ] = convert_uchar4(color * 255.0f);
}
"""

By default, this sample will run on the first platform and device it finds.

To choose a different OpenCL platform, simply change the platform index or device index to a different value.

In [ ]:
if __name__ == "__main__":
    platformIndex = 0
    deviceIndex = 0
    
    platforms = cl.get_platforms()
    print('Running on platform: ' + platforms[platformIndex].get_info(cl.platform_info.NAME))

    devices = platforms[platformIndex].get_devices()
    print('Running on device: ' + devices[deviceIndex].get_info(cl.device_info.NAME))

As before, we need an OpenCL context to work with and an OpenCL command queue to submit OpenCL commands to the OpenCL device, so create them.

In [ ]:
    context = cl.Context([devices[deviceIndex]])
    commandQueue = cl.CommandQueue(context, devices[deviceIndex])

Once we have an OpenCL context we can create an OpenCL program with the kernel string we created previously, build it, and get our Julia set kernel.

In [ ]:
    program = cl.Program(context, kernelString)
    program.build()
    kernel = program.Julia

We can also create a buffer to store our Julia set image.

In [ ]:
    deviceMemDst = cl.Buffer(context, cl.mem_flags.ALLOC_HOST_PTR, 
                             width * height * 4 * np.uint8().itemsize)

We are now ready to execute our Julia set kernel!

Unlike previous samples, this sample can either use a default `NULL` local work group size, or a specific local work group size.

In [ ]:
    iterations = 16
    lws = None

    print('Executing the kernel {} times'.format(iterations))
    print('Global Work Size = {}'.format([width, height]))
    if lws:
        print('Local Work Size = {}'.format(lws))
    else:
        print('Local Work Size = NULL')

For more stable timing we will execute the kernel multiple times.
By default, we will execute the kernel 16 times.
This may not be enough for some very fast devices.
If the timing is still unstable, please try increasing the number of iterations, or increasing the image size.

Note that we will call `clFinish` to ensure the OpenCL command queue is empty before we start the timer, and we will also call `clFinish` to make sure all commands have finished execution before ending the timer.
If we do not call `clFinish` in both places then we may measure the cost of commands that were still in the queue before the commands we want to time, or some commands may still be executing.

In [ ]:
    # Ensure the queue is empty and no processing is happening
    # on the device before starting the timer.
    commandQueue.finish()

    cr = -0.123
    ci = 0.745
    
    start = time.perf_counter()
    for i in range(iterations):
        kernel(commandQueue, [width, height], lws,
               deviceMemDst, np.float32(cr), np.float32(ci))

    # Ensure all processing is complete before stopping the timer.
    commandQueue.finish()

    end = time.perf_counter()
    print('Finished in {} seconds'.format(end - start))

All that's left to do now is to get the results of our Julia set kernel.
We can do this by mapping our output buffer, as usual.

In [ ]:
    mapped_dst, event = cl.enqueue_map_buffer(commandQueue, deviceMemDst,
                                              cl.map_flags.READ, 
                                              0, width * height, np.uint32)
    with mapped_dst.base:
        (r, g, b, a) = Image.fromarray(mapped_dst.reshape((width, height)), 'RGBA').split()
        image = Image.merge('RGB', (r, g, b))

Now we can display the results.

In [ ]:
        plt.imshow(image)

We can also save a bitmap for future offline viewing.

In [ ]:
        filename = 'julia.bmp'
        image.save(filename)
        print('Wrote image file {}'.format(filename))

Unlike prior samples, this sample can optionally specify a local work size grouping.  You can specify a local work size grouping by changing this line from:

```python
lws = None
```

to a specific local work size, for example:

```python
lws = [8, 8]
```

The local work size grouping is one way to tune an application for an architecture.
Can you find a local work size grouping that performs better than the implementation-determined grouping?
Is there a local work size grouping that performs very poorly on your implementation?
